## Using S3

S3 is a very simple object-oriented system that lets you define different behavior for functions, depending upon their input argument. This chapter explains how to use S3, and how generics and methods work.

### What's in a Name?
S3 uses a strict naming convention: all S3 methods have a name of the form generic.class.

The converse is not true: a function can have a name containing a dot without being an S3 method. This is the case with many of the functions that have been around since the early days of the S language. For example, all.equal() is actually an S3 generic, not a method. (This is an example of how leopard.case can be confusing.)

You can check if a function is an S3 generic by calling is_s3_generic() from the pryr package. You can also print it (by typing its name in the console), then looking to see if it calls UseMethod().

Similarly, you can check if a function is an S3 method by calling is_s3_method() from pryr. For example,

library(pryr)

is_s3_generic("t")           # generic transpose function

is_s3_method("t.data.frame") # transpose method for data.frames

is_s3_method("t.test")       # a function for Student's t-tests 

Which statements are true?

1. is.complex() is a method of the is generic that acts on complex objects.
2. seq.Date() is a method of the seq generic that acts on Date objects.
3. is.na.data.frame() is a method of the is.na generic that acts on data.frame objects.
4. sort() is a generic function.
5. order() is a generic function.

Answer: 2, 3 and 4

In [4]:
# install.packages("pryr")
library(pryr)
is_s3_method("is.complex")
is_s3_method("seq.Date")
is_s3_method("is.na.data.frame")
is_s3_generic("sort")
is_s3_generic("order")

[1] FALSE

[1] TRUE

[1] TRUE

[1] TRUE

[1] FALSE

### Creating a Generic Function
You can create your own S3 functions. The first step is to write the generic. This is typically a single line function that calls UseMethod(), passing its name as a string.

The first argument to an S3 generic is usually called x, though this isn't compulsory. It is also good practice to include a ... ("ellipsis", or "dot-dot-dot") argument, in case arguments need to be passed from one method to another.

Overall, the structure of an S3 generic looks like this.

an_s3_generic <- function(x, maybe = "some", other = "arguments", ...) {
  
  UseMethod("an_s3_generic")

}

In [5]:
# Create get_n_elements
get_n_elements <- function(x, ...)
{
  UseMethod("get_n_elements")
}

### Creating an S3 Method (1)
By itself, the generic function doesn't do anything. For that, you need to create methods, which are just regular functions with two conditions:

The name of the method must be of the form generic.class.
The method signature - that is, the arguments that are passed in to the method - must contain the signature of the generic.
The syntax is:

generic.class <- function(some, arguments, ...) {
  
    # Do something

}

In [9]:
# load datasets to get sleep data
library(datasets)

# View get_n_elements
get_n_elements

# Create a data.frame method for get_n_elements
get_n_elements.data.frame <- function(x,...){

return = nrow(x) * ncol(x)

}




# Call the method on the sleep dataset
n_elements_sleep <- get_n_elements.data.frame(sleep)

# View the result
n_elements_sleep

function(x, ...)
{
  UseMethod("get_n_elements")
}

[1] 60

### Creating an S3 method (2)
If no suitable method is found for a generic, then an error is thrown. For example, at the moment, get_n_elements() only has a method available for data.frames. If you pass a matrix to get_n_elements() instead, you'll see an error.

'> get_n_elements(matrix())

Error: no applicable method for 'get_n_elements' applied to an object of class "c('matrix', 'logical')"

Rather than having to write dozens of methods for every kind of input, you can create a method that handles all types that don't have a specific method. This is called the default method; it always has the name generic.default. For example, print.default() will print any type of object that doesn't have its own print() method.

In [12]:
# View predefined objects
ls.str()

# Create a default method for get_n_elements
get_n_elements.default <- function(x,...){

    length(unlist(x))
}

# Call the method on the ability.cov dataset
n_elements_ability.cov <- get_n_elements.default(ability.cov)

get_n_elements : function (x, ...)  
get_n_elements.data.frame : function (x, ...)  
get_n_elements.default : function (x, ...)  
n_elements_ability.cov :  int 43
n_elements_sleep :  int 60

### Finding Available Methods (1)
To find all the available methods for an S3 generic, call the methods() function.

It takes one argument that can be passed with or without quotes (though quoting is preferred, since the lookup feature to turn an R expression into a string is not perfect). This example shows both syntaxes, applied to the generic wilcox.text() for running Wilcoxon-Mann-Whitney rank sum tests. They give the same output:

methods("wilcox.test")

[1] wilcox.test.default* wilcox.test.formula*

see '?methods' for accessing help and source code

methods(wilcox.test)

[1] wilcox.test.default* wilcox.test.formula*

see '?methods' for accessing help and source code

In [13]:
# Find methods for print
methods("print")

  [1] print.acf*                                          
  [2] print.AES*                                          
  [3] print.anova*                                        
  [4] print.aov*                                          
  [5] print.aovlist*                                      
  [6] print.ar*                                           
  [7] print.Arima*                                        
  [8] print.arima0*                                       
  [9] print.AsIs                                          
 [10] print.aspell*                                       
 [11] print.aspell_inspect_context*                       
 [12] print.bibentry*                                     
 [13] print.Bibtex*                                       
 [14] print.browseVignettes*                              
 [15] print.by                                            
 [16] print.bytes*                                        
 [17] print.changedFiles*                               

### Method Lookup for Primitive Generics
Some core functionality of R is defined using primitive functions, which use a special technique for accessing C-code, for performance reasons. Examples of primitive functions include language elements, like if and for, operators like + and $, and mathematical functions like exp and sin. Primitive functions include S3 generics; the complete list of S3 primitive generics can be found using .S3PrimitiveGenerics.

When an S3 generic is primitive, its lookup process to find methods works slightly differently. R will look for methods using the class, as normal, but if nothing is found, the internal C-code function will be called. (Compare this to regular generics, where an error is thrown if no method is found.) This means that if you override the class of an object, fundamental behavior like calculating the length will not be broken.

In [19]:
# define a list
hair = list(colors = c("black", "brown", "blonde" , "ginger" , "grey"),  
            styles = c("afro", "beehive", "crew cut", "mohawk", "mullet", "pony tail"))
# check the class
class(hair)
# change the class
class(hair) <- "hairstylist"
# View the structure of hair
str(hair)
# What primitive generics are available?
.S3PrimitiveGenerics

# Does length.hairstylist exist?
exists("length.hairstylist()")

# What is the length of hair?
length(hair)





[1] "list"

List of 2
 $ colors: chr [1:5] "black" "brown" "blonde" "ginger" ...
 $ styles: chr [1:6] "afro" "beehive" "crew cut" "mohawk" ...
 - attr(*, "class")= chr "hairstylist"


[1] "anyNA"          "as.character"   "as.complex"     "as.double"     
 [5] "as.environment" "as.integer"     "as.logical"     "as.call"       
 [9] "as.numeric"     "as.raw"         "c"              "dim"           
[13] "dim<-"          "dimnames"       "dimnames<-"     "is.array"      
[17] "is.finite"      "is.infinite"    "is.matrix"      "is.na"         
[21] "is.nan"         "is.numeric"     "length"         "length<-"      
[25] "levels<-"       "names"          "names<-"        "rep"           
[29] "seq.int"        "xtfrm"

[1] FALSE

[1] 2

### Who is Calling?
is.na() is a primitive generic. In the console, look at its available methods.

When you call is.na(list(TRUE, FALSE, NA)), which of the following statements is true?

1. data.frames are lists, so is.na.data.frame() is called.

2. Neither is.na.list() nor is.na.default() exist, so an error will be thrown.

3. Since neither is.na.list() nor is.na.default() exist, the primitive is.na() is called.

4. lists are rasters, so is.na.raster() is called.

Answer 3

In [21]:
methods("is.na")

[1] is.na.data.frame      is.na.numeric_version is.na.POSIXlt        
[4] is.na.raster*        
see '?methods' for accessing help and source code

### Very Classy
Variables can have more than one class. In this case, class() returns a character vector of length greater than one.

Likewise you can set multiple classes by assigning a character vector to class(). The classes should be ordered from more specific to more general as you move left to right, since you want to begin with the behavior most targeted to your object. For example:

x <- c("a", "e", "i", "o", "u")

class(x) <- c("vowels", "letters", "character")

You can check for the other classes using the general purpose inherits() function. For example:

inherits(x, "vowels")

In [22]:
kitty <- "Miaow!"

# Assign classes
class(kitty) <- c("cat", "mammal", "character")

# Does kitty inherit from cat/mammal/character?
inherits(kitty, "cat")
inherits(kitty, "mammal")
inherits(kitty, "character")

# Is kitty a character vector?
is.character(kitty)

# Does kitty inherit from dog?
inherits(kitty, "dog")

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

[1] FALSE

### Writing the Next Method
When objects have multiple classes, you may wish to call methods for several of these classes. This is done using NextMethod().

The S3 methods now take the form:

an_s3_method.some_class <- function(x, ...){

  '# Act on some_class, then
  NextMethod("an_s3_method")
}

That is, NextMethod() should be the last line of the method.

In [24]:
# cat method
what_am_i.cat <- function(x, ...)
{
  # Write a message
  message("I'm a cat")
  # Call NextMethod
NextMethod("what_am_i")
}

# mammal method
what_am_i.mammal <- function(x, ...)
{
  # Write a message
  message("I'm a mammal")
  # Call NextMethod
NextMethod("what_am_i")
}
# character method
what_am_i.character <- function(x, ...)
{
  # Write a message
  message("I'm a character vector")
}
# Call what_am_i()
what_am_i(kitty)
# expected output
# what_am_i(kitty)
# I'm a cat
# I'm a mammal
# I'm a character vector

ERROR: Error in what_am_i(kitty): could not find function "what_am_i"
